# Analysis of the cleaned Frasier Transcript Dataset

This takes a look at the nature of the data, looking into how distinct the different dialects of each main character are

In [8]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [9]:
data = pd.read_csv("./data/cleanedTranscript.csv")
data.head(3)

,character,lines,season,episode,gender,title,directedBy,writtenBy,originalAirDate,viewershipInMillions,imdbVotes,imdbRatings,characterName,actorName,characterType,episodeCount
0,Frasier,listen to yourself bob you follow her to work ...,1,1,male,The Good Son,James Burrows,David Angell & Peter Casey & David Lee,1993-09-16,28.0,528,8.8,Frasier Crane,Kelsey Grammer,main,1.0
1,Roz,yes dr crane on line four we have russell from...,1,1,female,The Good Son,James Burrows,David Angell & Peter Casey & David Lee,1993-09-16,28.0,528,8.8,Roz Doyle,Peri Gilpin,main,1.0
2,Frasier,hello russell this is dr frasier crane im list...,1,1,male,The Good Son,James Burrows,David Angell & Peter Casey & David Lee,1993-09-16,28.0,528,8.8,Frasier Crane,Kelsey Grammer,main,1.0


In [10]:
def wordFrequencyChart(character: str):
    """This function makes a plot showing the word frequency of a specific character"""
    pass